# Sparse Embeddings

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import load_dataset
from qdrant_client import models, QdrantClient
from fastembed import SparseTextEmbedding

from src import load_data, embeddings


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]
c:\Users\manua\Documents\repos\dslabs\dslab-rag-retrieval\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\manua\AppData\Local\Temp\fastembed_cache\models--Qdrant--bm25. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
2025-09-03 16:35:58.053

# Params

In [7]:
INDEX = "ragquas-sparse-01"
RETRIEVE_K=5

# Environment variables

In [8]:
load_dotenv()


True

# Clients

In [9]:
client_qdrant = QdrantClient(":memory:") 


# Load Data

In [11]:
df_ragquas = load_data.ragquas()
df_ragquas.head(2)

Generating test split: 100%|██████████| 201/201 [00:00<00:00, 7550.54 examples/s]


,topic,answer,question,variant,context_1,context_2,context_3,context_4,context_5,link_1,link_2,link_3,link_4,link_5,text_1,text_2,text_3,text_4,text_5
0,reclamaciones,La opción más fácil y eficaz para reclamar una...,¿Cuál es la forma más fácil de reclamar cuando...,question_1,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,
1,reclamaciones,La opción más fácil y eficaz para reclamar una...,hola mi vuelo a Alemania se ha retrasado mucho...,question_2,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,


In [12]:
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


In [ ]:
documents = [
    "You should stay, study and sprint.",
    "History can only prepare us to be surprised yet again.",
]

encoder = SparseTextEmbedding(model_name="Qdrant/bm25")
sp_embeddings = list(encoder.embed(documents))


print(f"{sp_embeddings[0].values=}")
print(f"{sp_embeddings[0].indices=}")

embeddings[0].values=array([1.67868852, 1.67868852, 1.67868852])
embeddings[0].indices=array([1881538586,  150760872, 1932363795])


In [25]:
type(embeddings[0])

fastembed.sparse.sparse_embedding_base.SparseEmbedding

In [26]:
lst_embeddings = []

corpus_encoded = list(encoder.embed(df_docs['text_']))

for idx in range(len(corpus_encoded)):
    sp_vec = corpus_encoded[idx]
    lst_embeddings.append({
        "sparse": models.SparseVector(
            indices=sp_vec.indices,
            values=sp_vec.values
            )
        }
    )

print(f"{len(lst_embeddings)=}")

len(lst_embeddings)=183


In [30]:
lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs,
)
print(f"{len(lst_pts)=}")

len(lst_pts)=183


In [31]:
lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)
print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=183


In [35]:
client_qdrant.create_collection(
    collection_name=INDEX,
    vectors_config={},
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(),
        }
)

client_qdrant.upsert(
    collection_name=INDEX,
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [37]:
_q_text = df_docs.iloc[0:1]['question']

list(encoder.embed(documents))[0]


SparseEmbedding(values=array([1.67868852, 1.67868852, 1.67868852]), indices=array([1881538586,  150760872, 1932363795]))

In [ ]:
_q_text = df_docs.iloc[0:1]['question']

_q_encoded = next(encoder.embed(_q_text))



_q_vec = models.SparseVector(
    indices=_q_encoded.indices,
    values=_q_encoded.values
    )

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        INDEX,
        _q_vec,
        retrieve_k=3,
        using='sparse'  # passed by *args, **kwargs to query_points()
)



Question: 0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object
point.id='6b60567f-c238-46a4-bcc4-ca0ddf7ba48d'
point.score=0.24995039403438568
Doc: ¿Quieres estudiar japonés pero no puedes ir a una academia o tomar lecciones con un profesor particular? No te desanimes, porque en la actualidad hay muchos recursos para estudiar japonés de manera autodidacta. Así que guárdate este artículo en tu lista de favoritos para ir avanzando en el estudio del japonés.

Desde libros de gramática a páginas web con contenido gratuito y aplicaciones móviles para el estudio de kanji o vocabulario, las opciones son muchas y variadas. Por eso, a continuación t...
idx: 69, j: 1
------------------------------
